In [1]:
import fiiscraper as fscp
from fiiscraper.models import FII
import pandas as pd
import yfinance as yf
from datetime import date
from dateutil.relativedelta import relativedelta
from bs4 import BeautifulSoup
import requests

scraper = fscp.Scraper()

# Adiciona o sufixo .SA para ativos brasileiros
lista_fiis = scraper.listar_todos_fiis()
lista_reduzida = lista_fiis[:10]

#ticker = ["BCFF11", "YYYY11", "VISC11"]
df = scraper.buscar_precos_em_lote([fii.ticker for fii in lista_reduzida])

# Busca os dados de indicadores do dia
indicadores_fiis = []
for fii in lista_reduzida:

    # Busca indicadores do FII
    indicadores_fii = scraper.buscar_indicadores_dia(fii.ticker)
    
    # Adiciona FII à lista
    indicadores_fiis.append(indicadores_fii)

{'FII': 'ALZR11', 'Cotação': '10,50', 'Nome': 'ALIANZA TRUST RENDA IMOBILIÁRIA - FUNDO DE INVESTIMENTO IMOBILIÁRIO RESPONSABILIDADE LIMITADA', 'Data últ cot': '20/10/2025', 'Mandato': '', 'Min 52 sem': '8,59', 'Segmento': 'Multicategoria', 'Max 52 sem': '10,55', 'Gestão': 'Ativa', 'Vol $ méd (2m)': '1.547.460', 'Valor de mercado': '1.283.800.000', 'Nro. Cotas': '122.266.250', 'Relatório': '30/09/2025', 'Últ Info Trimestral': '30/06/2025', 'Dia': '0,03%', 'FFO Yield': '8,21%', 'FFO/Cota': '0,86', 'Mês': '-0,16%', 'Div. Yield': '9,3%', 'Dividendo/cota': '1,72', '30 dias': '0,89%', 'P/VP': '0,99', 'VP/Cota': '10,66', '12 meses': '11,90%', '2025': '13,15%', '2024': '-5,40%', 'Receita': '146.613.000', 'Receita_2': '41.225.900', '2023': '17,81%', 'Venda de ativos': '118.142.000', 'Venda de ativos_2': '51.541.000', '2022': '12,19%', 'FFO': '105.377.000', 'FFO_2': '25.329.500', '2021': '-4,29%', 'Rend. Distribuído': '210.500.000', 'Rend. Distribuído_2': '30.523.800', '2020': '-1,41%', '': '', 

In [3]:
lol = scraper._buscar_html('https://www.fundamentus.com.br/detalhes.php?papel=MXRF11')
scraper._parsear_pagina_fii(lol.text)

{}

In [6]:
%pip install pyarrow

  Using cached pyarrow-21.0.0-cp313-cp313-win_amd64.whl.metadata (3.4 kB)
Using cached pyarrow-21.0.0-cp313-cp313-win_amd64.whl (26.1 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import boto3
import pandas as pd
import io  # Necessário para o buffer em memória
import pyarrow
from datetime import date, timedelta

df = pd.DataFrame({'A':[1, 2, 3],
                   'B': [4, 5, 6]})

today = date.today() - timedelta(days=1)
yesterday = today

nome_bucket = 'fii-data-bucket'
nome_arquivo_s3_indicators = f'raw/daily_prices/price_date={yesterday.isoformat()}/data_parquet'
nome_arquivo_s3_prices = f'raw/price_history_snapshots/ingest_date={today.isoformat()}/data_parquet'

s3_client = boto3.client('s3')

# 1. Cria um buffer de bytes em memória (uma "caixa de despacho" virtual)
buffer_parquet = io.BytesIO()

# 2. Escreve o DataFrame no buffer em formato Parquet
df.to_parquet(buffer_parquet, index=False)

# 3. "Rebobina" o buffer para o início antes de o ler para o upload
buffer_parquet.seek(0)

# 4. Usa put_object para enviar o conteúdo do buffer (os bytes)
s3_client.put_object(
    Bucket=nome_bucket,
    Key=nome_arquivo_s3_indicators,
    Body=buffer_parquet
)

s3_client.put_object(
    Bucket=nome_bucket,
    Key=nome_arquivo_s3_prices,
    Body=buffer_parquet
)

{'ResponseMetadata': {'RequestId': 'CN8TMJ9RB8T8VTKF',
  'HostId': 'JBO3eNCXQjBy9Y38wLGzdJwc8jsgZCKshRVOTBYy6aKH1V2H1PChERiZps1O85Hb+03l4IftZU0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'JBO3eNCXQjBy9Y38wLGzdJwc8jsgZCKshRVOTBYy6aKH1V2H1PChERiZps1O85Hb+03l4IftZU0=',
   'x-amz-request-id': 'CN8TMJ9RB8T8VTKF',
   'date': 'Tue, 21 Oct 2025 12:14:53 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'x-amz-checksum-crc32': 'AAAAAA==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"',
 'ChecksumCRC32': 'AAAAAA==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}

,A,B
0,1,4
1,2,5
2,3,6
